In [ ]:
#Extra

#1 - Export environment variables

In [ ]:
#Main

#Import


In [ ]:
#complete_klines.py

#Import

In [13]:
#Db functions


#Import

import psycopg2
import psycopg2.extras
import os
from datetime import datetime
#from general_functions import log_handler

#Environment
'''
POSTGRES_DB = os.environ.get('POSTGRES_DB')
POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')
'''
#APAGAR####################################
DB_HOST='localhost'
POSTGRES_DB='pg_dev' 
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
POSTGRES_USER='openbot'
###########################################

#Functions

def create_table(table_name,keys,**kwargs):
    
    """Cria a tabela <table_name>, com as colunas <keys[i]>. Opcionalmente pode-se definir uma das chaves como 
    chave primária, ao invés da chave default atribuida pelo postgres.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser criada
    keys        -- Chaves (nomes das colunas)


    **kwargs:
    ========
    
    pk   -- Chave que será atribuida como primária 
    mute -- Suprime o log se 'yes'
    """
    
    pk = kwargs.get('pk')
    
    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    records = [] #Inicialmente definida como uma lista vazia
    
    create_table_job_status = 'fail' 
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM: create_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''.

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados    
    
                #Tenta criar a tabela
            try:
                
                sql = sql_command(table_name, keys, 'create')
                
                if (pk):
                    
                    sql_create_query = sql_command(table_name, keys, 'create', pk=pk)
                
                pointer.execute(sql_create_query)
                
                connection.commit()
                
                create_table_job_status = 'done'

            #Erro aocriar tabela
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                pointer.execute("ROLLBACK")

                msg = '''
FROM: create_table
''' + str(datetime.utcnow()) + '''

Não foi possível criar tabela ''' + table_name + ''' devido ao erro: 
''' + str(error) + '''.

'''
   
                log_handler(msg,default_log)
    
    return create_table_job_status

                    
def read_table(table_name, **kwargs):
    
    """Busca no banco a tabela [table_name], retornando uma lista, tal que cada elemento representa
    uma linha (registro) da tabela, ou uma lista vazia, caso a tabela não seja encontrada. Pode retornar
    a tabela inteira, ou parte dos valores se forem informados, corretamente, os argumentos opcionais
    **kwargs.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    
    field_key -- Chave (nome da coluna) do campo tomado como referência de ordenamento
    
    sort_type      -- Modo como os registros são buscados:
        
    *sort_type options*
     -----------------
        - DESC ==> Do maior para o menor
        - ASC  ==> Do menor para o maior
        
    limit     -- Número de registros que devem ser retornados
    """
    
    field_key = kwargs.get('field_key')
    
    sort_type = kwargs.get('sort_type')
    
    limit = kwargs.get('limit')

    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    records = [] #Inicialmente definida como uma lista vazia
    
    default_log = 'print'

    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Conectado ao banco com sucesso!

'''

        log_handler(msg,default_log)

    #Não conecta ao banco    
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta obter a tabela de pares
            try:
                
                sql_select_query = 'SELECT * FROM ' + table_name
                
                if (field_key and sort_type and limit):
                    
                    sql_select_query = sql_select_query + ' ORDER BY ' + field_key + ' ' + sort_type + ' LIMIT ' + limit

                pointer.execute(sql_select_query)

                records = pointer.fetchall()
                
                msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name +''' obtida com sucesso! 

'''
                
                log_handler(msg,default_log)

            #Não encontra tabela de pares
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
                
                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Não foi possível obter tabela ''' + table_name +''' devido ao erro: 
''' + str(error) + '''

'''
                
                log_handler(msg,default_log)
            
            pointer.close()
            connection.close()
    
    return records


def update_table(table_name, pk_field, pk_value, field_to_update, new_field_value, **kwargs):
  
    """Dada uma certa tabela, atualiza uma entrada específica da mesma.
    
    
    Keyword arguments:
    =================
    
    table_name      -- Nome da tabela a ser buscada
    pk_field        -- Chave (nome da coluna) atribuída como chave primária; ``pk``, se chave default.
    pk_value        -- Valor da chave primária do registro
    field_to_update -- Chave (nome da coluna) do campo a ser regravado
    new_field_value -- Novo valor a ser gravado no campo
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    """
    
    mute = kwargs.get('mute')

    connection = ''
    
    update_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM: update_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta atualizar entrada na tabela de pares
            try:

                sql_update_query = 'Update ' + table_name + ' set ' + field_to_update + ' = %s where ' + pk_field + '= %s'
                
                pointer.execute(sql_update_query, (new_field_value, pk_value))

                connection.commit()

                update_table_job_satus = 'done'

            #Não consegue atualizar entrada
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                #Desfaz a ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: update_table
''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name + ''': Não foi possível atualizar a entrada ''' + field_to_update + ''' 
do registro ''' + pk_value + ''' para ''' + new_field_value + ''' devido ao erro: 
''' + str(error) + '''

'''
   
                log_handler(msg,default_log)
    
            pointer.close()
            connection.close()

    return update_table_job_satus


def save_in_table(table_name, keys, data, **kwargs):
    
    """Salva na tabela <table_name>, nas respectivas colunas <keys[i]>, os dados correspondentes, <data>.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela na qual os dados devem ser gravados
    keys        -- Chaves (nomes das colunas)
    data        -- dados a serem gravados


    **kwargs:
    ========

    mute -- Suprime o log se 'yes'
    """

    mute = kwargs.get('mute')

    connection = ''
    
    save_in_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM: save_in_table
''' + str(datetime.utcnow()) + '''

Não foi possível salvar na tabela ''' + + ''' devido ao erro: 
''' + str(error) + '''

'''
        
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            pass
            #Tenta atualizar entrada na tabela de pares
            #try:
    
    
    
    
    
    return save_in_table_job_satus

In [2]:
#binance functions

#Import

In [3]:
#Other functions

#Import

import sys
import os
import notification

def log_handler(msg,destination):
    
    """Envia uma mensagem de log para o destino escolhido pelo usuário.
    
    
    Keyword arguments:
    =================
    
    msg         -- Mensagem de log
    destination -- Caminho padrão para saída da mensagem de log
    
    *destination options*
     -------------------
        - mute   ==> Ignora a mensagem
        - print  ==> Printa na tela
        - logm   ==> Envia para o arquivo padrão de monitoramento
        - logw   ==> Envia para o arquivo padrão de alertas
        - tlgrm  ==> Envia para o grupo de monitoramento do telegram
        - tlgrw  ==> Envia para o grupo de alertas do telegram    
    """
    
    if (destination == 'mute'):
        
        pass
    
    elif (destination == 'print'):
        
        print(msg)
    
    elif (destination == 'logm'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_monitoring.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'logw'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_warning.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'tlgrm'):
        
        notification.Telegrambot(msg).monitoring()
        
    elif (destination == 'tlgrw'):
        
        notification.Telegrambot(msg).warning()


def sql_command(table_name, keys, action, **kwargs):
    
    """Dado um nome de tabela <table_name> e o dicionário <keys> (em que a chave será o nome da coluna e 
    o valor definirá o tipo de dado gravado - vide documentação do PostgreSQL), retorna o comando SQL ade-
    quado para criação da tabela ou salvamento de dados em uma tabela.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    keys        -- Dicionário com as chaves e seus respectivos tipos
    action      -- O tipo de ação desejada com o comando SQL, criar ou salvar.

    *action options*
     ------------
        - create ==> SQL para criar uma tabela
        - save   ==> SQL para salvar em uma tabela
    
    **kwargs:
    =========
    
    pk -- Chave primária
    """
    
    pk = kwargs.get('pk')
    
    key_list = list(keys.keys())
    type_list = list(keys.values())
    
    sql = ''
    
    if (action == 'create'):
        
        sql = 'CREATE TABLE ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            entry = key_list[i] + ' ' + type_list[i]
            
            if (pk):
                
                if (key_list[i] == pk):
                    
                    entry = key_list[i] + ' ' + type_list[i] + ' PRIMARY KEY'
                        
            sql = sql + entry + ', '

        sql = sql + key_list[len(keys)-1] + ' ' + type_list[len(keys)-1] + ')'
        
    if (action == 'save'):
        
        format_code = ''
        
        sql = 'INSERT INTO ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            sql = sql + key_list[i] + ' ' + ', '
            
            format_code = format_code + '%s, '

        sql = sql + key_list[len(keys)-1] + ' ' + ') VALUES (' + format_code + '%s)'
                        
    return sql        

# Zona de testes

## read_table

In [ ]:
#Coletando todas as amostras

table_name = 'binance_pairs'

pairs = read_table(table_name)

if (len(pairs) > 0):
    
    for pair in pairs:
        
        print (pair)

In [ ]:
#Coletando uma parte das amostras

table_name = 'binance_pairs'

candles = read_table('binance_klines_1m_btcusdt', field_key = 'open_time', mode = 'DESC', limit = '5')

if (len(candles) > 0):
    
    for candle in candles:
        
        print (candle)

## update_table

In [19]:
#Testando para mudança da chave 'status', da tabela 'binance_pairs' Modifique À vontade.

#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================


table_name = 'binance_pairs'

pk_field = 'name'

pk_value = 'BTCUSDT'

field_to_update = 'status'

new_field_value = randomString(8) #Máximo 8 caracteres

records = read_table(table_name, mute='yes')

for record in records:

    if (record[pk_field] == pk_value):

        old_record = record
        
        old_field_value = record[field_to_update]

update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, new_field_value)

print(update_table_job_status + '''
''')

if (update_table_job_status == 'done'):
    
    records = read_table(table_name, mute='yes')
    
    for record in records:
        
        if (record[pk_field] == pk_value):
            
            new_record = record
            
    #Desfazendo
    update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, old_field_value)
    
print('''Entrada anterior   - ''' + str(old_record) + '''
Entrada atualizada - ''' + str(new_record) + '''
''')

done

Entrada anterior   - ['BTCUSDT', 'OFF', 'absent', None, None]
Entrada atualizada - ['BTCUSDT', 'OFF', 'vzelynbe', None, None]



In [14]:
table_name = 'biance_pairs'

pk_field = 'name'

pk_value = 'BTCUSDT'

field_to_update = 'status'

new_field_value = 'test' #Máximo 8 caracteres

update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, new_field_value)
print(update_table_job_status)


FROM: update_table
2019-06-08 20:37:33.049630

Tabela biance_pairs: Não foi possível atualizar a entrada status 
do registro BTCUSDT para test devido ao erro: 
relation "biance_pairs" does not exist
LINE 1: Update biance_pairs set status = 'test' where name= 'BTCUSDT...
               ^



fail


## create_table

In [ ]:
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))


recent_created_table_name = randomString(10)

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

create_table_job_satus = create_table(recent_created_table_name,keys,pk='open_time')

print(create_table_job_satus)

## save_in_table
